Użyj każdą z tych funkcji 
* `unix_timestamp()` 
* `date_format()`
* `to_unix_timestamp()`
* `from_unixtime()`
* `to_date()` 
* `to_timestamp()` 
* `from_utc_timestamp()` 
* `to_utc_timestamp()`

In [0]:
from pyspark.sql.functions import current_date, current_timestamp

kolumny = ["timestamp", "unix", "Date"]
dane = [("2015-03-22T14:13:34", 1646641525847, "May, 2021"),
        ("2015-03-22T15:03:18", 1646641557555, "Mar, 2021"),
        ("2015-03-22T14:38:39", 1646641578622, "Jan, 2021")]

dataFrame = spark.createDataFrame(dane, kolumny) \
    .withColumn("current_date", current_date()) \
    .withColumn("current_timestamp", current_timestamp())

display(dataFrame)

timestamp,unix,Date,current_date,current_timestamp
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-18,2025-03-18T09:04:48.818+0000
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-18,2025-03-18T09:04:48.818+0000
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-18,2025-03-18T09:04:48.818+0000


In [0]:

dataFrame.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- unix: long (nullable = true)
 |-- Date: string (nullable = true)
 |-- current_date: date (nullable = false)
 |-- current_timestamp: timestamp (nullable = false)



## unix_timestamp(..) & cast(..)

Konwersja **string** to a **timestamp**.

Lokalizacja funkcji 
* `pyspark.sql.functions` in the case of Python
* `org.apache.spark.sql.functions` in the case of Scala & Java

## 1. Zmiana formatu wartości timestamp yyyy-MM-dd'T'HH:mm:ss 
`unix_timestamp(..)`

Dokumentacja API `unix_timestamp(..)`:
> Convert time string with given pattern (see <a href="http://docs.oracle.com/javase/tutorial/i18n/format/simpleDateFormat.html" target="_blank">SimpleDateFormat</a>) to Unix time stamp (in seconds), return null if fail.

`SimpleDataFormat` is part of the Java API and provides support for parsing and formatting date and time values.

2. Zmień format zgodnie z klasą `SimpleDateFormat`**yyyy-MM-dd HH:mm:ss**
  * a. Wyświetl schemat i dane żeby sprawdzicz czy wartości się zmieniły

In [0]:
from pyspark.sql.functions import col, to_timestamp, unix_timestamp
zmianaFormatu = dataFrame.withColumn("timestamp", to_timestamp(col("timestamp")))

zmianaFormatu.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- unix: long (nullable = true)
 |-- Date: string (nullable = true)
 |-- current_date: date (nullable = false)
 |-- current_timestamp: timestamp (nullable = false)



In [0]:
#unix_timestamp
tempE = zmianaFormatu.withColumn("timestamp",unix_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss"))
display(tempE)

timestamp,unix,Date,current_date,current_timestamp
1427033614,1646641525847,"May, 2021",2025-03-18,2025-03-18T09:07:43.331+0000
1427036598,1646641557555,"Mar, 2021",2025-03-18,2025-03-18T09:07:43.331+0000
1427035119,1646641578622,"Jan, 2021",2025-03-18,2025-03-18T09:07:43.331+0000


## Stwórz nowe kolumny do DataFrame z wartościami year(..), month(..), dayofyear(..)

In [0]:
#date_format
from pyspark.sql.functions import date_format
yearDate = dataFrame.withColumn("year", date_format(col("current_timestamp"), "yyyy")).withColumn("month", date_format(col("current_timestamp"), "MM")).withColumn("DayOfYear", date_format(col("current_timestamp"), "DD")) 
display(yearDate)

timestamp,unix,Date,current_date,current_timestamp,year,month,DayOfYear
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-18,2025-03-18T09:09:25.666+0000,2025,03,77
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-18,2025-03-18T09:09:25.666+0000,2025,03,77
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-18,2025-03-18T09:09:25.666+0000,2025,03,77


In [0]:
#to_date()
from pyspark.sql.functions import to_date, year, month, dayofyear
toDate = dataFrame.withColumn("year", year(to_date(col("current_timestamp")))).withColumn("month", month(to_date(col("current_timestamp")))).withColumn("DayOfYear", dayofyear(to_date(col("current_timestamp"))))
display(toDate)

timestamp,unix,Date,current_date,current_timestamp,year,month,DayOfYear
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-18,2025-03-18T09:10:33.168+0000,2025,3,77
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-18,2025-03-18T09:10:33.168+0000,2025,3,77
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-18,2025-03-18T09:10:33.168+0000,2025,3,77


In [0]:
#from_unixtime()
from pyspark.sql.functions import from_unixtime
fromUnix = dataFrame.withColumn("date", from_unixtime(col("unix"), "yyyy-MM-dd HH:mm:ss.SSS")).withColumn("year", year(col("date"))) \
    .withColumn("month", month(col("date"))) \
    .withColumn("dayofyear", dayofyear(col("date")))
display(fromUnix)

timestamp,unix,date,current_date,current_timestamp,year,month,dayofyear
2015-03-22T14:13:34,1646641525847,+54149-12-28 23:50:47.000,2025-03-18,2025-03-18T09:19:21.786+0000,54149,12,362
2015-03-22T15:03:18,1646641557555,+54149-12-29 08:39:15.000,2025-03-18,2025-03-18T09:19:21.786+0000,54149,12,363
2015-03-22T14:38:39,1646641578622,+54149-12-29 14:30:22.000,2025-03-18,2025-03-18T09:19:21.786+0000,54149,12,363


In [0]:
#to_timestamp()
from pyspark.sql.functions import to_timestamp
toTimestamp =  dataFrame.withColumn("timestamp", to_timestamp(col("timestamp"))).withColumn("year", year(col("timestamp"))) \
    .withColumn("month", month(col("timestamp"))) \
    .withColumn("dayofyear", dayofyear(col("timestamp")))
display(toTimestamp)


timestamp,unix,Date,current_date,current_timestamp,year,month,dayofyear
2015-03-22T14:13:34.000+0000,1646641525847,"May, 2021",2025-03-18,2025-03-18T09:18:54.609+0000,2015,3,81
2015-03-22T15:03:18.000+0000,1646641557555,"Mar, 2021",2025-03-18,2025-03-18T09:18:54.609+0000,2015,3,81
2015-03-22T14:38:39.000+0000,1646641578622,"Jan, 2021",2025-03-18,2025-03-18T09:18:54.609+0000,2015,3,81


In [0]:
#to_utc_timestamp()
from pyspark.sql.functions import to_utc_timestamp
toUtcTimestamp = dataFrame \
    .withColumn("timestampUTC", to_utc_timestamp(col("timestamp"), "PST")) \
    .withColumn("year", year(col("timestampUTC"))) \
    .withColumn("month", month(col("timestampUTC"))) \
    .withColumn("dayofyear", dayofyear(col("timestampUTC")))
display(toUtcTimestamp)



timestamp,unix,Date,current_date,current_timestamp,timestampUTC,year,month,dayofyear
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-18,2025-03-18T09:17:29.099+0000,2015-03-22T21:13:34.000+0000,2015,3,81
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-18,2025-03-18T09:17:29.099+0000,2015-03-22T22:03:18.000+0000,2015,3,81
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-18,2025-03-18T09:17:29.099+0000,2015-03-22T21:38:39.000+0000,2015,3,81


In [0]:
#from_utc_timestamp()
from pyspark.sql.functions import from_utc_timestamp
fromUtcTimestamp = toUtcTimestamp \
    .withColumn("fromUTC", from_utc_timestamp(col("timestampUTC"), "PST")) \
    .withColumn("year", year(col("fromUTC"))) \
    .withColumn("month", month(col("fromUTC"))) \
    .withColumn("dayofyear", dayofyear(col("fromUTC")))
display(fromUtcTimestamp)

timestamp,unix,Date,current_date,current_timestamp,timestampUTC,year,month,dayofyear,fromUTC
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-18,2025-03-18T09:17:56.540+0000,2015-03-22T21:13:34.000+0000,2015,3,81,2015-03-22T14:13:34.000+0000
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-18,2025-03-18T09:17:56.540+0000,2015-03-22T22:03:18.000+0000,2015,3,81,2015-03-22T15:03:18.000+0000
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-18,2025-03-18T09:17:56.540+0000,2015-03-22T21:38:39.000+0000,2015,3,81,2015-03-22T14:38:39.000+0000


In [0]:
%fs ls dbfs:/databricks-datasets/
